Run this cell to load pyomo

In [ ]:
# Run this cell only if you are using Google Colab. 

# install required dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install -q pyomo
  !pip install mosek
  !apt-get install -y -qq glpk-utils
  !apt-get install -y -qq coinor-cbc
  !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
  !unzip -o -q ipopt-linux64
  !pip install ttictoc

     |████████████████████████████████| 9.7 MB 14.7 MB/s 
     |████████████████████████████████| 49 kB 2.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.4 MB 10.2 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1

This piece of code linearizes Pacejka's model but might be computationally expensive

In [ ]:
def PacejkaLinearizer(D,C,B,E,alpha_bar):
  x = sym.Symbol('x')
  Fy = D*sym.sin(C*sym.atan(B*x-E*(B*x-sym.atan(B*x))))
  Fy_bar= Fy.evalf(subs=alpha_bar)
  dfy_alph = sym.diff(Fy,x)
  c_bar = dfy_alph.evalf(subs = alpha_bar)

  return Fy_bar, c_bar, alpha_bar

The below piece of code linearizes Dugoff's Tire model allowing for a choice


In [ ]:
def DugoffLinearizer(slip_angle,slip_ratio, ca, cs, mu, Fz):
  x = sym.Symbol('x')
  lamb_bar = mu*Fz*(1+slip_ratio)/(2*((cs*slip_ratio)^2+(ca*slip_angle)^2))
  lamb = mu*Fz*(1+slip_ratio)/(2*((cs*slip_ratio)^2+(ca*x)^2))

  if lamb_bar < 1:
    f = lamb*(2-lamb)
  else:
    f = 1
  
  Fy = ca*x*f/(1+slip_ratio)
  Fy_bar= Fy.evalf(subs=slip_angle)
  dfy_alph = sym.diff(Fy,x)
  c_bar = dfy_alph.evalf(subs = slip_angle)

  return Fy_bar, c_bar, slip_angle

Vehicle model Block

In [ ]:
import numpy as np
from scipy.linalg import block_diag
import sympy as sym
import mosek
import pyomo.environ as pyo
# States: y,v,psi, r, phi, phi_dot
# Inputs: Change in each wheel torque and in each wheel steer angle

# Vehicle Parameters: 
reff = 0.1905
lf = 1.6566
lr = 1.3152
l = lf+lr
tf = 1.638762
tr = 1.5239686
m = 803.182 #kg
mf = 782/2.2
mr = 985/2.2
mufw = 29.5
murw = 36
ms = m - 2*(mufw+murw)
ixx = 200 
hs = 0 #considered CG and roll center to be same point ##############
iy = 800
iz = 1200
g = 9.81
kus_d = mf /Caf +mr /Car #Caf and Car needed


#####################################################
################### Cphi no value ###################
################### Kphi no value ###################
#####################################################


# Linearize tyre model at equilibrium point
def CFTOC_VD(u, Phir, W0,q,r,muy, mux, psid, yd, alphar_max, fz_0, fy_0, Q_0, delta1):
  fy1, ca1, a1 = PacejkaLinearizer(D1,C1,B1,E1,alpha_bar1)
  fy2, ca2, a2 = PacejkaLinearizer(D2,C2,B2,E2,alpha_bar2)
  fy3, ca3, a3 = PacejkaLinearizer(D3,C3,B3,E3,alpha_bar3)
  fy4, ca4, a4 = PacejkaLinearizer(D4,C4,B4,E4,alpha_bar4)

  B11 = np.array([[0,0,0,0,0,0], [0,-ca1/u,0, -lf*ca1/u, 0, 0]])
  B21 = np.array([[1/reff, 0],[0, ca1]])
  D11 = np.array ([0, fy1-ca1*a1])

  B12 = np.array([[0,0,0,0,0,0], [0,-ca2/u,0, -lf*ca2/u, 0, 0]])
  B22 = np.array([[1/reff, 0],[0, ca2]])
  D12 = np.array ([0, fy2-ca2*a2])

  B13 = np.array([[0,0,0,0,0,0], [0,-ca3/u,0, -lr*ca3/u, 0, 0]])
  B23 = np.array([[1/reff, 0],[0, ca3]])
  D13 = np.array ([0, fy3-ca3*a3])

  B14 = np.array([[0,0,0,0,0,0], [0,-ca4/u,0, -lr*ca4/u, 0, 0]])
  B24 = np.array([[1/reff, 0],[0, ca4]])
  D14 = np.array ([0, fy4-ca4*a4])

  #Local actuator reconfiguration matrix
  tq1 = 0
  tq2 = 0
  tq3 = 1
  tq4 = 1
  tdelta1 = 1
  tdelta2 = 1
  tdelta3 = 0
  tdelta4 = 0

  Tw1 = np.array([[tq1,0],[0,tdelta1]])
  Tw2 = np.array([[tq2,0],[0,tdelta2]])
  Tw3 = np.array([[tq3,0],[0,tdelta3]])
  Tw4 = np.array([[tq4,0],[0,tdelta4]])

  #Reconfiguration (Mapping) Matricies: Delta1 input
  Lw1 = np.array ([[np.cos(delta1), -np.sin(delta1)],[np.sin(delta1), np.cos(delta1)]])
  delta2 = delta1
  Lw2 = np.array ([[np.cos(delta2), -np.sin(delta2)],[np.sin(delta2), np.cos(delta2)]])
  Lw3 = np.array([[1,0],[0,1]])
  Lw4 = np.array([[1,0],[0,1]])

  #Mapping Matrix from corner forces
  Lc = np.array([[1,0,1,0,1,0,1,0],[0,1,0,1,0,1,0,1], [-tf/2,lf,tf/2,lf,-tr/2,-lr,tr/2,-lr]])

  # Vehicle Body Dynamics y vy psi,psidot roll, rollrate 
  Af = np.array([[0, 1, 0, 0, 0, 0],[0,0,0 -u, ms*hs*(kphi-ms*g*hs)/(-ms**2*hs**2+m*ixx), ms*hs*cphi/(-ms**2*hs**2+m*ixx)], [0,0, 0, 1, 0, 0],[0,0,0,0, 0, 0],[0,0,0,0,0,1],[0,0,0, 0, -m*(kphi-ms*g*hs)/(-ms**2*hs**2+m*ixx), -m*cphi/(-ms**2*hs**2+m*ixx)]])
  Bf  = np.array([[0,0,0],[0, ixx/(-ms**2*hs**2+m*ixx), 0],[0,0,0],[0,0,1/iz],[0,0,0],[0, ms*hs/(-ms**2*hs**2+m*ixx),0]])
  Cpr = np.array([0],[(ms*hs*(ms*hs*g-kphi)-ms*g*ixx)/(m*ixx-(ms*hs)**2)],[0],[0],[0],[(m*kphi+ms**2*hs*g-g*hs*m*ms)/(m*ixx-(ms*hs)**2)]) #assume small phi-phir

  # X_dot = AX+EW+BU+D+FPhir
  # W = wheel torque, steer angle for each wheel (8x1)
  # States are v r phi, phi_dot (ignoring tyre dynamics)
  # U = change in wheel torque, change in steering angle for each tire

  Tw = block_diag(Tw1,Tw2,Tw3,Tw4)
  Lw = block_diag(Lw1, Lw2, Lw3, Lw4)
  B1 = np.array([B11.T, B12.T, B13.T, B14.T]).T
  B2 = block_diag(B21, B22, B23, B24)
  D1 = np.array([D11.T,D12.T,D13.T,D14.T]).T

  A = Af+Bf*Lc*Lw*B1

  E = Bf*Lc*Lw*B2

  B = Bf*Lc*Lw*B2*Tw

  D = Bf*Lc*Lw*D1 

  F = Cpr


  rmax = muy*9.81/u
  rb = u/(l+kus_d*u**2)*delta1
  rd = np.sign(delta1)*np.min([np.abs(rb), rmax])
  xd = np.array([yd, 0, psid, rd, 0 ,0])

  
  model = pyo.ConcreteModel()
  model.tidx = pyo.Set(initialize=range(0, N+1)) # length of finite optimization problem
  model.xidx = pyo.Set(initialize=range(0, nx))
  model.uidx = pyo.Set(initialize=range(0, nu))
  model.phiridx = pyo.Set(initialize = range(0,nphir))

  model.x = pyo.Var(model.xidx, model.tidx)
  model.u = pyo.Var(model.uidx, model.tidx)
#Objective
  objective_terms = [
      sum(q[i]*(model.x[i,t]-xd[i])**2 for i in model.xidx for t in model.tidx if t<N+1),
      sum(r[i]*(model.u[i,t])**2 for i in model.uidx for t in model.tidx if t<N)
  ]

  model.Obj = pyo.Objective(expr = sum(objective_terms), sense = pyo.minimize)

  #Dynamic constraints
  model.constraint1 = pyo.Constraint(model.xidx, model.tidx, rule = lambda model, i, t : model.x[i,t+1] == model.x[i,t] + Ts*(A[i,:]@model.x[:,t] + E[i,:]@model.w[:,t] + B[i,:]@model.u[:,t] +D[i]+F[i]*Phir)
                                  if t < N+1 else pyo.Constraint.Skip)
  model.constraint2 = pyo.Constraint(model.uidx, model.tidx, rule = lambda model, i, t: model.u[2*i,t] <= mux*fz_0[i]*np.sqrt(1-(fy_0[i]/muy/fz_0[i])**2)*reff-Q_0[i] 
                                      if (t<N and i<=3) else pyo.Constraint.Skip)
  model.constraint3 = pyo.Constraint(model.tidx, rule = lambda model, t: -model.x[1,t]/u+lr/u*model.x[3,t] < alphar_max if t< N-1 else pyo.Constraint.Skip)
  model.constraint4 = pyo.Constraint(model.tidx, rule = lambda model, t: +model.x[1,t]/u-lr/u*model.x[3,t] > alphar_max if t< N-1 else pyo.Constraint.Skip)
  model.constraint5 = pyo.Constraint(model.uidx, model.tidx, rule = lambda model, i, t: -model.u[2*i,t] >= mux*fz_0[i]*np.sqrt(1-(fy_0[i]/muy/fz_0[i])**2)*reff-Q_0[i] 
                                      if (t<N and i<=3) else pyo.Constraint.Skip)
  model.constraint6 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[1,t]== model.u [3,t] 
                                     if t<N else pyo.Constraint.Skip)
  model.constraint7 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[4,t]== model.u [6,t] 
                                     if t<N else pyo.Constraint.Skip)
  model.constraint8 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[1,t] <= deltamax] 
                                     if t<N else pyo.Constraint.Skip)
  model.constraint8 = pyo.Constraint(model.tidx, rule = lambda model, t: model.u[1,t] >= -deltamax] 
                                     if t<N else pyo.Constraint.Skip)
  ## model.constraint9 = pyo.Constraint(model.tidx, rule = lambda model, t: model.x[0,t] == yd[])



    

NameError: ignored

MPC Problem Definition



Using a QP solution But i dont like this

In [ ]:
def Sx_Su_Sw_Sd_Sf(A, B, E, D,F, N):
    
    nX = np.size(A,0)
    nU = np.size(B,1)
    nD = np.size(D,1)
    nE = np.size(E,1)

    Sx = np.eye(nX)
    A_tmp = A
    for i in range(N):
        Sx = np.vstack((Sx, A_tmp))
        A_tmp = A_tmp @ A
        
    SxB = Sx @ B
    Su = np.zeros((nX*(N+1),nU*N))
    for j in range(N):
        Su_tmp = np.vstack((np.zeros((nX, nU)), SxB[:-nX,:]))
        Su[:, j] = Su_tmp.reshape(Su_tmp.shape[0], )
        SxB = Su_tmp
        
    SxE = Sx @ E
    Sw = np.zeros((nX*(N+1),nE*N))
    for k in range(N):
        Sw_tmp = np.vstack((np.zeros((nX, nE)), SxE[:-nX,:]))
        Sw[:, j] = Sw_tmp.reshape(Sw_tmp.shape[0], )
        SxE = Sw_tmp

    Sd = np.zeros(nX)
    D_tmp = D
    for i in range(N):
        Sd = np.vstack((Sd, D_tmp))
        D_tmp = D_tmp @ A

    Sxf = Sx @ F
    Sf = np.zeros((nX*(N+1),1*N))
    for j in range(N):
        Sf_tmp = np.vstack((np.zeros((nX, 1)), SxB[:-nX,:]))
        Sf[:, j] = Sf_tmp.reshape(Sf_tmp.shape[0], )
        Sxf = Sf_tmp
    
    return Sx, Su, Sw, Sd, Sf

In [ ]:
N = 10 #sample horizon length
nx = 6
nu = 8
nW = 8
nphir =1

Ts = 0.1 #placeholder for sampling time

# In the paper they use zero order hold, I will use euler discretization
Ad = Ts*A + np.eye(np.size(A,0))
Bd = Ts*B
Ed = Ts*E
Dd = Ts*D
Fd = Ts*F

#Formulating Problem:
## NOT SURE IF WE NEED THESE YET ##

Sx, Su, Sw, Sd, Sf = Sx_Su_Sw_Sd_Sf(Ad, Bd, Ed, Dd, Fd, N)

# We need a weighting matrix Q. I would say emphasis on tracking lateral position, then heading, then lateral velocity, with little concern for roll charact.
Q = np.array([10,5,8,2,0,0]) # TO BE DEFINED LATER
PN = Q #TO BE DEFINED LATER
R = np.array(np.ones((1,8))) #TO BE DEFINED LATER

Q_temp = np.kron(np.eye(N),Q)
Qbar  = block_diag(Q_temp,PN)

R =[] # TO BE DEFINED LATER
R_temp = np.kron(np.eye(N), R)
Rbar = block_diag(R_temp)

Wbar = np.kron(np.eye(N), W0) #W0 recieved from vehicle, vector of inputs at time zero

H = Su.T@Qbar@Su + Rbar
F1 = Sx.T @Qbar @ Su
F2 = Sw.T @ Qbar @ Su
F3 = Sd.T@Q@Su
F4 = - Qbar@Su
F5 = Sf.T@Qbar@Su

# Finally Cost Matricies:

P = H
q = x0.T@F1+Wbar.T@F2 +F3+ PhirBar@F5+ xdes.T@F4 #x0 from vehicle, xdes from path planner

# Constraints to add:
# Final waypoint is reached 
# Initial position is as recieved by vehicle at that time
# Slip control
# Yaw rate control
# Actuator constraints